In [9]:
import numpy as np
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from textblob import TextBlob
from wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
%matplotlib inline

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
cf.go_offline()

import plotly.graph_objs as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)


In [10]:
df=pd.read_csv("amazon.csv")

In [11]:
df.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4,No issues.,23-07-2014,138,0,0,0,0,0.0,0.0
1,1,0mie,5,"Purchased this for my device, it worked as adv...",25-10-2013,409,0,0,0,0,0.0,0.0
2,2,1K3,4,it works as expected. I should have sprung for...,23-12-2012,715,0,0,0,0,0.0,0.0
3,3,1m2,5,This think has worked out great.Had a diff. br...,21-11-2013,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5,"Bought it with Retail Packaging, arrived legit...",13-07-2013,513,0,0,0,0,0.0,0.0


In [12]:
if 'unnamed: 0' in df.columns:
    df.drop('unnamed: 0', inplace=True, axis=1)
if 'wilson_lower_bound' in df.columns:
    df = df.sort_values("wilson_lower_bound", ascending=False)

print(df.head())

      Unnamed: 0             reviewerName  overall  \
2031        2031     Hyoun Kim "Faluzure"        5   
3449        3449        NLee the Engineer        5   
4212        4212              SkincareCEO        1   
317          317  Amazon Customer "Kelly"        1   
4672        4672                  Twister        5   

                                             reviewText  reviewTime  day_diff  \
2031  [[ UPDATE - 6/19/2014 ]]So my lovely wife boug...  05-01-2013       702   
3449  I have tested dozens of SDHC and micro-SDHC ca...  26-09-2012       803   
4212  NOTE:  please read the last update (scroll to ...  08-05-2013       579   
317   If your card gets hot enough to be painful, it...  09-02-2012      1033   
4672  Sandisk announcement of the first 128GB micro ...  03-07-2014       158   

      helpful_yes  helpful_no  total_vote  score_pos_neg_diff  \
2031         1952          68        2020                1884   
3449         1428          77        1505                1

In [6]:
def check_dataframe(df, head=5, tail=5):
    print("SHAPE".center(82, '~'))
    print(f'Rows: {df.shape[0]}')
    print(f'Columns: {df.shape[1]}')
    print("TYPES".center(82, '~'))
    print(df.dtypes)
    print("".center(82, '~'))
    # Remove this line if you don't need missing values analysis
    # print(missing_values_analysis(df))  
    print('DUPLICATE VALUES'.center(83, '~'))
    print(df.duplicated().sum())
    print("QUANTILES".center(82, '~'))
    print(df.select_dtypes(include=[np.number]).quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)
    
check_dataframe(df)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~SHAPE~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Rows: 4915
Columns: 12
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~TYPES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Unnamed: 0                int64
reviewerName             object
overall                   int64
reviewText               object
reviewTime               object
day_diff                  int64
helpful_yes               int64
helpful_no                int64
total_vote                int64
score_pos_neg_diff        int64
score_average_rating    float64
wilson_lower_bound      float64
dtype: object
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~DUPLICATE VALUES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~QUANTILES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                       0.00   0.05    0.50         0.95        0.99  \
Unnamed: 0              0.0  245.7  2457.0  4668.300000  4864.86000   
overall            

In [8]:
def check_class(dataframe):
    nunique_df = pd.DataFrame({
        'Variable': dataframe.columns,
        'Classes': [dataframe[col].nunique() for col in dataframe.columns]
    })
    nunique_df = nunique_df.sort_values('Classes', ascending=False)
    nunique_df = nunique_df.reset_index(drop=True)
    return nunique_df

check_class(df)



,Variable,Classes
0,Unnamed: 0,4915
1,reviewText,4912
2,reviewerName,4594
3,reviewTime,690
4,day_diff,690
5,wilson_lower_bound,40
6,score_average_rating,28
7,score_pos_neg_diff,27
8,total_vote,26
9,helpful_yes,23
